In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas import set_option
import time
from numpy import set_printoptions
from numpy.random import random, uniform, choice
from numpy import set_printoptions, int0
from numpy import (zeros, min, log10, tanh, arctanh,
                    shape, clip, exp, pi, cos,sqrt, ravel, log)
from copy import deepcopy
import typing as t
from sklearn.linear_model import LinearRegression
from typing import Any

# <div style="color:blue">MULTIPLE REGRESSION</div>

In [2]:
# File Paths 
compression = "/home/adisa/Desktop/Working/Akinwale/compression.csv"
flexural = "/home/adisa/Desktop/Working/Akinwale/flexural.csv"
split = "/home/adisa/Desktop/Working/Akinwale/split.csv"


In [4]:
def linear_regression(path: str) -> Any:
    p = path.split("/")[-1].split(".")[0]
    df_com = pd.read_csv(path)
    df_des= df_com.describe()
    lb =  df_des.iloc[3, :-1].values
    ub = df_des.iloc[-1,:-1].values
    X = df_com.iloc[:, 1:-1].values
    model_x = df_com.iloc[::9, 1: -1].values
    Y = df_com.iloc[:, -1].values
    model = LinearRegression().fit(X, Y)
    r_sq = model.score(X, Y)
    inter = model.intercept_
    coeff = model.coef_
    print(p)
    statement = f"b0:{round(inter, 2)}, b1:{round(coeff[0], 2)}, b2:{round(coeff[1], 2)}, b3:{round(coeff[2], 2)}, b4:{round(coeff[3], 2)}, b5:{round(coeff[4], 2)}"
    return X, lb, ub

In [5]:
###################
# Compression, flexural and split
lr_com = linear_regression(compression)
lr_flex = linear_regression(flexural)
lr_split = linear_regression(split)

compression
flexural
split


In [6]:
xc, lb_c, ub_c = lr_com
xf, lb_f, ub_f = lr_flex
xs, lb_s, ub_s = lr_split

# <div style="color:green">Memory</div>

In [7]:
def memory(pol_size, d):
    lb = [0.113, 0, 0.457, 1.371, 0.113]
    ub = [1.029, 0.206, 2.286, 6.857, 0.566]
    Matrix_Harmony_Memory = np.zeros((pol_size,d))
    #print(Matrix_Harmony_Memory)
    for i in range(pol_size):
        for j in range(d):
            Matrix_Harmony_Memory[i,j] =lb[j] + np.random.random()*(ub[j]-lb[j])
    return Matrix_Harmony_Memory

In [9]:
X_memory = memory(1000, 5)
lb = [0.113, 0, 0.457, 1.371, 0.113]
ub = [1.029, 0.206, 2.286, 6.857, 0.566]

# <div style="color:green">META_HEURISTIC ALGORITHMS</div>

## <div style="color:yellow">CONSTANTS</div>


In [14]:
# harmony      
HMCR = 0.95                   #Harmony memory consideration rate
PAR = 0.3                     #Pitch Adjustment rate
bw = 1                        #bandwidth
# Fire-fly
ALPHA = 1 # randomness 
GAMMA = 0.01 #absorption coefficient
BETA0 = 1 
THETA= 0.97
LOLP = 0.1 # adjustable
# slime mould
EPSILON = 10E-10
ID_FIT = 1                   #id of fittness 
ID_WEI = 2                   #id of weight
ID_POS = 0                   # id of best position normal
ID_BEST_F= 0                 # best position fittness
ID_WORST_F = -1              # worst position
Z = 0.03
# general
num_iter = 1000 #number of iterations
runs = 5

# <div style="color:grey">OBJECTIVE FUNCTIONS</div>

In [10]:
from math import ceil
def obj_func_compression(x):
    x = np.array(x).tolist()
    ce, sba, fi, co, wa = x
    comp = 34.81+409.54*ce+648.19*sba+0*fi-34.15*co+1.07*wa
    return comp

def obj_func_flexural(x):
    x = np.array(x).tolist()
    ce, sba, fi, co, wa = x
    flexural = 19.49-0.14*ce-8.84*sba+0.6*fi-1.19*co+0.15*wa
    return flexural

def obj_func_split(x):
    x = np.array(x).tolist()
    ce, sba, fi, co, wa = x
    split = 54.88-4.28*ce-320.83*sba+0*fi-6*co+0*wa
    return split
    
def generic_func(x):
     x = np.array(x).tolist()
     ce, sba, fi, co, wa = x
     res = abs(34.81+409.54*ce+648.19*sba+0*fi-34.15*co+1.07*wa)+ \
            abs(19.49-0.14*ce-8.84*sba+0.6*fi-1.19*co+0.15*wa)+ \
            abs(54.88-4.28*ce-320.83*sba+0*fi-6*co+0*wa)
     return res


# <div style="color:grey">FIRE FLY</div>

In [15]:
def fire_fly_search(lb: list, ub: list, harmony_mem: np.array, obj_func: t.Callable):
    start = time.time()
    global_best_score = {}
    for run in range(runs):
                
        Matrix_Flies_Memory = harmony_mem
        pol_size = len(Matrix_Flies_Memory)
        old_score = []
        for _ in range(pol_size):
            item = Matrix_Flies_Memory[_,:]
            score = obj_func(np.ravel(item))
            old_score.append(score)


        best_score = []
        components = []
        times = []
        for num in range (num_iter):
            # iteration looping
            
            scale = np.array(ub) - np.array(lb) # define stigma
            for i in range(pol_size):
                for j in range (pol_size):
                
                    
                    xi = Matrix_Flies_Memory[i,:] #main fly
                    xj = Matrix_Flies_Memory[j,:] # corresponding flies
                    
                    # checking the optimization condition 
                    if obj_func(xi) < obj_func(xj):
                        Matrix_Flies_Memory[i,:] = Matrix_Flies_Memory[i,:]

                    else:
                        # r in square
                        r = np.square(Matrix_Flies_Memory[i,:] - Matrix_Flies_Memory[i,:])

                        beta = BETA0 * np.exp(-GAMMA *r) #attractiveness
                        
                        steps = ALPHA * (np.random.random(1)[0]-0.5) *scale
                        
                        xnew = xi + beta *(xi - xj) + steps #new flies created
                        
                        xnew = np.clip(xnew, lb, ub)
                        #checking the boundaries with respect to inital boundaries
                        
                            
                    fnew = obj_func(xnew) #new function
                    f = obj_func(Matrix_Flies_Memory[i,:])
                    if fnew < f:
                        Matrix_Flies_Memory[i,:] = xnew
            min_obj = np.min([obj_func(Matrix_Flies_Memory[i,:]) for i in range(pol_size)])
            best_score.append(min_obj)
            components.append(xnew)
            times.append(time.time() - start)
            print (f'Number of Iteration: {num}, Best cost: {min_obj}, Components: {xnew}')
        print("========================================================")
        global_best_score[f'best_score_run_{run}'] = best_score
        global_best_score[f'components_run_{run}'] = components
        global_best_score[f'times_run_{run}'] = times
    return Matrix_Flies_Memory, global_best_score

In [16]:
gen_res = fire_fly_search(lb,ub, X_memory, generic_func)

Number of Iteration: 0, Best cost: 14.993995623710402, Components: [0.20968631 0.11364139 0.65005596 4.82254088 0.43181108]
Number of Iteration: 1, Best cost: 14.178903424570656, Components: [0.113      0.07900902 0.457      3.90024377 0.35565348]
Number of Iteration: 2, Best cost: 14.178903424570656, Components: [0.113      0.05975634 0.457      3.38752443 0.31331629]
Number of Iteration: 3, Best cost: 14.178903424570656, Components: [0.113      0.0270541  0.457      2.51662872 0.2414031 ]
Number of Iteration: 4, Best cost: 14.178903424570656, Components: [0.113      0.06996367 0.457      3.6593564  0.33576249]
Number of Iteration: 5, Best cost: 14.178903424570656, Components: [0.113      0.05099542 0.457      3.15421166 0.29405076]
Number of Iteration: 6, Best cost: 13.338025720142541, Components: [0.113      0.06441595 0.457      3.51161488 0.32356291]
Number of Iteration: 7, Best cost: 13.338025720142541, Components: [0.113      0.03433033 0.457      2.85830647 0.24791154]
Number o

In [ ]:
gen_res

In [ ]:
compression_fly_search = fire_fly_search(lb_c, ub_c, xc, obj_func_compression)

Number of Iteration: 0, Best cost: 34.43330277588871, Components: [0.11314288 0.         0.45714289 1.37142889 0.11314289]
Number of Iteration: 1, Best cost: 34.43330277588871, Components: [0.15786802 0.01987784 0.45714289 1.37142889 0.11314289]
Number of Iteration: 2, Best cost: 34.43330277588871, Components: [0.14172313 0.01270233 0.45714289 1.37142889 0.11314289]
Number of Iteration: 3, Best cost: 34.43330277588871, Components: [0.14045998 0.01214093 0.45714289 1.37142889 0.11314289]
Number of Iteration: 4, Best cost: 34.43330277588871, Components: [0.11314288 0.         0.45714289 1.37142889 0.11314289]
Number of Iteration: 5, Best cost: 34.43330277588871, Components: [0.12368929 0.00468729 0.45714289 1.37142889 0.11314289]
Number of Iteration: 6, Best cost: 34.43330277588871, Components: [0.13636051 0.01031895 0.45714289 1.37142889 0.11314289]
Number of Iteration: 7, Best cost: 34.43330277588871, Components: [0.1507953  0.01673441 0.45714289 1.37142889 0.11314289]
Number of Iterat

In [ ]:
compression_fly_search

(array([[0.11314288, 0.        , 0.45714289, 1.37142889, 0.11314289],
        [0.11314288, 0.        , 0.45714289, 1.37142889, 0.11314289],
        [0.11314288, 0.        , 0.45714289, 1.37142889, 0.11314289],
        [0.11314288, 0.        , 0.45714289, 1.37142889, 0.11314289],
        [0.11314288, 0.        , 0.45714289, 1.37142889, 0.11314289],
        [0.11314288, 0.        , 0.45714289, 1.37142889, 0.11314289],
        [0.11314288, 0.        , 0.45714289, 1.37142889, 0.11314289],
        [0.11314288, 0.        , 0.45714289, 1.37142889, 0.11314289],
        [0.11314288, 0.        , 0.45714289, 1.37142889, 0.11314289],
        [0.11314288, 0.        , 0.45714289, 1.37142889, 0.11314289],
        [0.11314288, 0.        , 0.45714289, 1.37142889, 0.11314289],
        [0.11314288, 0.        , 0.45714289, 1.37142889, 0.11314289],
        [0.11314288, 0.        , 0.45714289, 1.37142889, 0.11314289],
        [0.11314288, 0.        , 0.45714289, 1.37142889, 0.11314289],
        [0.11314288,

In [ ]:
split_fly_search = fire_fly_search(lb_s, ub_s, xs, obj_func_split)

NameError: name 'runs' is not defined

In [ ]:
split_fly_search

(array([[0.32091433, 0.06418287, 0.71314289, 2.13942889, 0.17650289],
        [0.32091433, 0.06418287, 0.71314289, 2.13942889, 0.17650289],
        [0.32091433, 0.06418287, 0.71314289, 2.13942889, 0.17650289],
        [0.32091433, 0.06418287, 0.71314289, 2.13942889, 0.17650289],
        [0.32091433, 0.06418287, 0.71314289, 2.13942889, 0.17650289],
        [0.32091433, 0.06418287, 0.71314289, 2.13942889, 0.17650289],
        [0.32091433, 0.06418287, 0.71314289, 2.13942889, 0.17650289],
        [0.32091433, 0.06418287, 0.71314289, 2.13942889, 0.17650289],
        [0.32091433, 0.06418287, 0.71314289, 2.13942889, 0.17650289],
        [0.32091433, 0.06418287, 0.71314289, 2.13942889, 0.17650289],
        [0.32091433, 0.06418287, 0.71314289, 2.13942889, 0.17650289],
        [0.32091433, 0.06418287, 0.71314289, 2.13942889, 0.17650289],
        [0.32091433, 0.06418287, 0.71314289, 2.13942889, 0.17650289],
        [0.32091433, 0.06418287, 0.71314289, 2.13942889, 0.17650289],
        [0.32091433,

In [ ]:
flexural_fly_search = fire_fly_search(lb_f, ub_f, xf, obj_func_flexural)

Number of Iteration: 0, Best cost: 10.823770933999999, Components: [0.57951337 0.13563938 2.28571444 6.85714333 0.56571433]
Number of Iteration: 1, Best cost: 10.823770933999999, Components: [0.70749611 0.12134459 2.28571444 6.85714333 0.56571433]
Number of Iteration: 2, Best cost: 10.823770933999999, Components: [0.80261945 0.10529118 2.28571444 6.85714333 0.56571433]
Number of Iteration: 3, Best cost: 10.823770933999999, Components: [1.02857144 0.20571429 2.28571444 6.85714333 0.56571433]
Number of Iteration: 4, Best cost: 10.823770933999999, Components: [0.97246502 0.1807781  2.28571444 6.85714333 0.56571433]
Number of Iteration: 5, Best cost: 10.823770933999999, Components: [1.02857144 0.20571429 2.28571444 6.85714333 0.56571433]
Number of Iteration: 6, Best cost: 10.823770933999999, Components: [1.02857144 0.20571429 2.28571444 6.85714333 0.56571433]
Number of Iteration: 7, Best cost: 10.823770933999999, Components: [0.83165921 0.11819774 2.28571444 6.85714333 0.56571433]
Number o

In [ ]:
flexural_fly_search

(array([[1.02857144, 0.20571429, 2.28571444, 6.85714333, 0.56571433],
        [1.02857144, 0.20571429, 2.28571444, 6.85714333, 0.56571433],
        [1.02857144, 0.20571429, 2.28571444, 6.85714333, 0.56571433],
        [1.02857144, 0.20571429, 2.28571444, 6.85714333, 0.56571433],
        [1.02857144, 0.20571429, 2.28571444, 6.85714333, 0.56571433],
        [1.02857144, 0.20571429, 2.28571444, 6.85714333, 0.56571433],
        [1.02857144, 0.20571429, 2.28571444, 6.85714333, 0.56571433],
        [1.02857144, 0.20571429, 2.28571444, 6.85714333, 0.56571433],
        [1.02857144, 0.20571429, 2.28571444, 6.85714333, 0.56571433],
        [1.02857144, 0.20571429, 2.28571444, 6.85714333, 0.56571433],
        [1.02857144, 0.20571429, 2.28571444, 6.85714333, 0.56571433],
        [1.02857144, 0.20571429, 2.28571444, 6.85714333, 0.56571433],
        [1.02857144, 0.20571429, 2.28571444, 6.85714333, 0.56571433],
        [1.02857144, 0.20571429, 2.28571444, 6.85714333, 0.56571433],
        [1.02857144,